[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/instant-ngp-colab/blob/main/instant_ngp.ipynb)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!apt-get install \
    cmake \
    libgoogle-glog-dev \
    libgflags-dev \
    libatlas-base-dev \
    libeigen3-dev \
    libsuitesparse-dev \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libboost-test-dev \
    libfreeimage-dev \
    libmetis-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev

!wget https://github.com/camenduru/instant-ngp-colab/releases/download/v1.0/ceres-solver.zip
!unzip /content/ceres-solver.zip -d ceres-solver

!cp -r /content/ceres-solver/lib/. /usr/local/lib
!chmod 755 /content/ceres-solver/bin/colmap
!cp -r /content/ceres-solver/bin/. /usr/local/bin

!wget https://github.com/camenduru/instant-ngp-colab/releases/download/v1.0/instant-ngp-v1.zip
!unzip /content/instant-ngp-v1.zip -d instant-ngp

!pip install commentjson huggingface-hub
%cd /content/instant-ngp
import os
from huggingface_hub import create_repo, upload_folder

In [ ]:
scene_path = "/content/drive/MyDrive/fox" #@param {type:"string"}
train_path = "/content/train"
if not os.path.isdir(scene_path):
  raise NotADirectoryError(scene_path)
!rm -rf {train_path}
!mkdir {train_path}
!cp -r {scene_path}/. /content/train
transforms_path = os.path.join(train_path, f"transforms.json")
!python ./scripts/colmap2nerf.py --colmap_matcher exhaustive --run_colmap --aabb_scale 4 --images {train_path} --out {transforms_path}

train_steps = 2000  #@param {type:"integer"}
snapshot_path = os.path.join(train_path, f"{train_steps}.msgpack")
!python ./scripts/run.py --scene {train_path} --mode nerf --n_steps {train_steps} --save_snapshot {snapshot_path}

In [ ]:
hf_token = "" #@param {type:"string"}
repo_id = "camenduru/instant-ngp-models" #@param {type:"string"}
path_in_repo = "fox" #@param {type:"string"}
create_repo(repo_id, private=True, token=hf_token)
upload_folder(folder_path=train_path, path_in_repo=path_in_repo, repo_id=repo_id, commit_message=f"{scene_path}", token=hf_token)